In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=655).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes1406.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model2705/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [144]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("desc.csv", sep=';', names=['id', 'label'])
ids = pd.Series(dscr['id'])
ids = ids.astype(str)
dscr['id'] = ids
ids = []
for i, id in enumerate(dscr['id']):
    if len(id) == 4:
        ids.append(str(id))
    else:
        ids.append('0' + str(id))
dscr['id'] = ids

In [183]:
#ищем классы нейросети, к которым нет описания.
withlabels = list(dscr['id'])
justids = list(Label_encoder.classes_)
for each in justids:
    if each not in withlabels:
        print(each)

0391
0481
0830
1012
1018
1062
1068
5022
5032
5035
5052
5058
5075
6908


In [148]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [149]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704': 'Кондитерские изделия из сахара (включая белый шоколад), не содержащие какао'}

In [150]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704'

In [151]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [159]:
#Пример с 3 классами
print(predict_prob('трубы сантехнические пластиковые ', 3))

{'3917': 0.9508989453315735, '7304': 0.011040955781936646, '4009': 0.0049907136708498}


In [153]:
#Пример с 5 классами
print(predict_prob('трубы металлические ', 5))

{'7304': 0.9171484112739563, '7306': 0.014556351117789745, '7411': 0.009647378697991371, '3917': 0.004930668044835329, '8307': 0.0029827230609953403}


In [155]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        result.append([each, dscr[dscr['id']==each].iloc[0]['label'], probs[each]])
    return result

In [158]:
#Пример с 5 классами.
predict_prob_with_descr('трубы металлические')

[['7304',
  'Трубы, трубки и профили полые, бесшовные, из черных металлов (кроме чугунного литья)',
  0.9171484112739563],
 ['7306',
  'Трубы, трубки и профили полые прочие (например, с открытым швом или сварные, клепаные или соединенные аналогичным способом), из черных металлов',
  0.014556351117789745],
 ['7411', 'Трубы и трубки медные', 0.009647378697991371],
 ['3917',
  'Трубы, трубки, шланги и их фитинги (например, соединения, колена, фланцы), из пластмасс',
  0.004930668044835329],
 ['8307',
  'Трубы гибкие из недрагоценных металлов, с фитингами или без них',
  0.0029827230609953403]]

In [157]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('трубы металлические'), columns=['ID', 'LABEL', 'PROB'])

,ID,LABEL,PROB
0,7304,"Трубы, трубки и профили полые, бесшовные, из ч...",0.917148
1,7306,"Трубы, трубки и профили полые прочие (например...",0.014556
2,7411,Трубы и трубки медные,0.009647
3,3917,"Трубы, трубки, шланги и их фитинги (например, ...",0.004931
4,8307,"Трубы гибкие из недрагоценных металлов, с фити...",0.002983


In [186]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [187]:
consolelog(predict_prob_with_descr("БЕЗАЛКОГОЛЬНЫЕ ГАЗИРОВАННЫЕ НАПИТКИ АРОМАТИЗИРОВАННЫЕ", 5))

prob-ty:    Code:    Description
0.65559:    2106:    Пищевые продукты, в другом месте не поим...
0.02286:    2103:    Продукты для приготовления соусов и гото...
0.01718:    1905:    Хлеб, мучные кондитерские изделия, пирож...
0.01501:    9602:    Обработанные материалы растительного или...
0.01474:    1704:    Кондитерские изделия из сахара (включая ...
